In [1]:
from tqdm.notebook import tqdm

In [2]:
%cd ..

from hydra import initialize, compose

initialize(config_path='../configs')

from src.configuration import register_configs

register_configs()

#cfg = compose(overrides=['+datamodule@dm=sl_datamodule', '+split_seed=0'])
cfg = compose(overrides=['+datamodule@dm=sl_datamodule', '+split_seed=2', 'datamodule/splits@dm.splits=uva600'])

from omegaconf import OmegaConf
OmegaConf.resolve(cfg)

from hydra.utils import instantiate

cfg.dm.splits.merge_test_centers = True
cfg.dm.splits.merge_val_centers = True
cfg.dm.splits.undersample_benign_train = True
cfg.dm.splits.undersample_benign_eval = True
cfg.dm.loader_config.num_workers = 0
cfg.dm.loader_config.balance_classes_train = True

from rich import print as pprint
pprint(OmegaConf.to_object(cfg))

/home/paulw/projects/TRUSnet-1


/tmp/ipykernel_10818/442535702.py:5: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path='../configs')
/home/paulw/anaconda3/envs/exact/lib/python3.9/site-packages/pl_bolts/utils/warnings.py:30: UserWarning: You want to use `gym` which is not installed yet, install it with `pip install gym`.
  stdout_func(


{
    'dm': ExactPatchDMConfig(
        _target_='src.lightning.datamodules.exact_datamodule.PatchDataModuleForSupervisedLear
ning',
        root='/home/mgilani/data/Exact/Exact',
        loader_config=LoaderConfig(
            batch_size=32,
            num_workers=0,
            balance_classes_train=True,
            train_strategy_ddp=False
        ),
        minimum_involvement=0.4,
        splits=SplitsConfig(
            _target_='src.data.exact.splits.Splits',
            cohort_specifier=['UVA600'],
            train_val_split_seed=0,
            train_val_ratio=0.1,
            test_as_val=False,
            undersample_benign_train=True,
            undersample_benign_eval=True,
            benign_cores_selection_seed=0,
            merge_train_centers=True,
            merge_val_centers=True,
            merge_test_centers=True
        ),
        patch_view_config=PatchViewConfig(
            patch_size=[5, 5],
            patch_strides=[1, 1],
            subpatch_size=[1, 1],
            needle_region_only=True,
            prostate_region_only=False,
            prostate_intersection_threshold=0.9,
            needle_intersection_threshold=0.6
        ),
        patch_transform_train=TransformConfig(
            _target_='src.data.exact.transforms.TransformV3',
            norm=NormConfig(
                _target_='src.data.exact.transforms.Normalize',
                mode='instance',
                type='min-max',
                truncate=True
            ),
            tensor_transform=None,
            us_augmentation=None,
            out_size=[256, 256]
        ),
        patch_transform_eval=TransformConfig(
            _target_='src.data.exact.transforms.TransformV3',
            norm=NormConfig(
                _target_='src.data.exact.transforms.Normalize',
                mode='instance',
                type='min-max',
                truncate=True
            ),
            tensor_transform=None,
            us_augmentation=None,
            out_size=[256, 256]
        )
    ),
    'split_seed': 2
}

In [3]:
dm = instantiate(cfg.dm)
dm.setup()

Preparing cores:   0%|          | 0/353 [00:00<?, ?it/s]/home/paulw/projects/TRUSnet-1/src/data/exact/core.py:176: UserWarning: You appear to be calling Core constructor manually. It is preferable to call `Core.create_core(specifier)`
                    which will cache core instances and not double memory requirements for core objects!
                
  warn(
Preparing cores:   0%|          | 0/40 [00:00<?, ?it/s]/home/paulw/projects/TRUSnet-1/src/data/exact/core.py:176: UserWarning: You appear to be calling Core constructor manually. It is preferable to call `Core.create_core(specifier)`
                    which will cache core instances and not double memory requirements for core objects!
                
  warn(
Preparing cores:   0%|          | 0/81 [00:00<?, ?it/s]/home/paulw/projects/TRUSnet-1/src/data/exact/core.py:176: UserWarning: You appear to be calling Core constructor manually. It is preferable to call `Core.create_core(specifier)`
                    which will cache 

In [4]:
next(iter(dm.train_dataloader()))

PatchesDatasetOutput(patch=tensor([[[[0.6528, 0.6674, 0.6769,  ..., 0.6704, 0.6771, 0.6874],
          [0.4039, 0.4211, 0.4323,  ..., 0.3877, 0.3919, 0.3984],
          [0.3894, 0.3682, 0.3544,  ..., 0.4304, 0.4106, 0.3803],
          ...,
          [0.5174, 0.5300, 0.5382,  ..., 0.3468, 0.3391, 0.3273],
          [0.4975, 0.4911, 0.4868,  ..., 0.4934, 0.4583, 0.4047],
          [0.4558, 0.4330, 0.4180,  ..., 0.4850, 0.4918, 0.5021]]],


        [[[0.5353, 0.5505, 0.5605,  ..., 0.4998, 0.5009, 0.5027],
          [0.5539, 0.5526, 0.5517,  ..., 0.4945, 0.4921, 0.4886],
          [0.4952, 0.4855, 0.4791,  ..., 0.4911, 0.4885, 0.4845],
          ...,
          [0.4684, 0.4566, 0.4489,  ..., 0.4521, 0.4523, 0.4526],
          [0.4634, 0.4634, 0.4633,  ..., 0.4983, 0.4969, 0.4947],
          [0.4941, 0.5104, 0.5211,  ..., 0.5818, 0.5820, 0.5822]]],


        [[[0.3953, 0.4018, 0.4060,  ..., 0.0492, 0.0000, 0.0000],
          [0.2757, 0.3211, 0.3509,  ..., 0.5476, 0.5774, 0.6231],
          [

In [15]:
from src.modeling.seq_models import core_classifier_batch_to_output
from src.modeling.registry import core_classifier_no_pos_emb, core_classifier

model = core_classifier_no_pos_emb()

In [16]:
from src.utils.metrics import ClassificationOutputCollector
from itertools import product
from src.utils.metrics import recall, specificity, find_best_threshold, f1
from torchmetrics.functional import auroc

metrics = []

def get_metrics(out): 
    return {
        'auroc': auroc(out['preds'], out['labels'], num_classes=2), 
        'sens': recall(out),
        'spec': specificity(out),
    }

for seed, split_seed in product(range(4), range(4)): 
    coll = ClassificationOutputCollector()
    model = core_classifier(seed, split_seed)
    model.eval().to('cuda:1')
    for batch in tqdm(dm.test_dataloader()): 
        out = core_classifier_batch_to_output(model, batch, 'cuda:1')
        coll.collect_batch(out['classification_output'])

    out = coll.compute()
    metrics.append(get_metrics(out))

  0%|          | 0/315 [00:00<?, ?it/s]

  0%|          | 0/315 [00:00<?, ?it/s]

  0%|          | 0/315 [00:00<?, ?it/s]

  0%|          | 0/315 [00:00<?, ?it/s]

  0%|          | 0/315 [00:00<?, ?it/s]

...

  0%|          | 0/315 [00:00<?, ?it/s]

  0%|          | 0/315 [00:00<?, ?it/s]

  0%|          | 0/315 [00:00<?, ?it/s]

  0%|          | 0/315 [00:00<?, ?it/s]

  0%|          | 0/315 [00:00<?, ?it/s]

In [24]:
def map_dict(func, dict): 
    return {
        k: func(v) for k, v in dict.items()
    }
metrics = [map_dict(lambda t: t.item(), d) for d in metrics]

In [25]:
import pandas as pd 
pd.DataFrame(metrics).describe()

,auroc,sens,spec
count,16.000000,16.000000,16.000000
mean,0.802837,0.879849,0.512249
std,0.019626,0.115764,0.146397
min,0.744325,0.500000,0.301508
25%,0.799027,0.857759,0.419598
50%,0.805255,0.900862,0.512563
75%,0.817265,0.933190,0.594221
max,0.824164,1.000000,0.869347


In [9]:
from src.utils.metrics import recall, specificity, find_best_threshold, f1
from torchmetrics.functional import auroc

find_best_threshold(lambda t: f1(out, t))


0.631578947368421

In [10]:
auroc(out['preds'], out['labels'], num_classes=2)

tensor(0.8213)

In [12]:
recall(out, 0.631)

tensor(0.8017)

In [13]:
specificity(out, t=0.631)

tensor(0.6533)

In [12]:
out

{'pool_output': tensor([[-0.5660, -1.0687,  0.7693,  0.4849,  0.6054, -1.5200,  0.0223,  1.2418,
           0.2667,  1.4233, -1.8840, -0.7201,  0.5774, -0.7001,  0.9728, -1.2168,
           0.7124,  0.9373, -0.4822, -1.5839,  1.2310, -0.8357,  1.6772,  0.8965,
          -0.7231, -0.6984,  0.7920, -1.4582, -0.5311, -1.9888,  0.8943,  1.8770,
           0.1428,  0.1381, -1.0102, -1.3770,  0.7585, -0.1336,  0.5614,  0.7186,
          -0.0615,  1.6559,  0.4951, -0.5544, -0.5137,  0.2699, -0.0854,  1.9345,
          -1.1790, -0.5267, -1.2091, -0.6821,  0.8250, -0.8055, -1.5858, -0.6206,
          -0.7375, -0.7987,  1.3566,  1.4024,  0.7109,  0.6846,  0.4298,  0.3947]]),
 'logits': tensor([[-0.4904,  0.8790]]),
 'preds': tensor([[0.2027, 0.7973]]),
 'labels': tensor([1]),
 'patient_id': tensor([603]),
 'path_on_server': array(['/med-i_data/Data/Exact_Ultrasound/data/full_data/GS9/UVA-0603_LML_GS9.mat'],
       dtype='<U73'),
 'center': array(['UVA'], dtype='<U3'),
 'loc': array(['LML'], dtyp